# TASK 0: INITIALIZE THE LLM CLIENT

In [42]:
import openai

openai.api_key = "voc-1682785691266774337744690b15dbca9a82.58870878"
openai.api_base = "https://openai.vocareum.com/v1"

# Define the LLM Model
MODEL = "gpt-3.5-turbo"

# TASK 1: CREATE THE CSV

In [43]:
!pip install pandas

Defaulting to user installation because normal site-packages is not writeable


In [44]:
# All import statements used in this task
from pydantic import BaseModel, Field, NonNegativeInt
from typing import List
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
import pandas as pd

In [45]:
# Example Instruction and Sample
INSTRUCTION = "Generate a CSV file with at least 10 real estate listing."
SAMPLE_LISTING = \
"""
Neighborhood: Green Oaks
Price: $800,000
Bedrooms: 3
Bathrooms: 2
House Size: 2,000 sqft

Description: Welcome to this eco-friendly oasis nestled in the heart of Green Oaks. This charming 3-bedroom, 2-bathroom home boasts energy-efficient features such as solar panels and a well-insulated structure. Natural light floods the living spaces, highlighting the beautiful hardwood floors and eco-conscious finishes. The open-concept kitchen and dining area lead to a spacious backyard with a vegetable garden, perfect for the eco-conscious family. Embrace sustainable living without compromising on style in this Green Oaks gem.

Neighborhood Description: Green Oaks is a close-knit, environmentally-conscious community with access to organic grocery stores, community gardens, and bike paths. Take a stroll through the nearby Green Oaks Park or grab a cup of coffee at the cozy Green Bean Cafe. With easy access to public transportation and bike lanes, commuting is a breeze.
"""

In [46]:
class RealEstate(BaseModel):
    """The structure of the Real Estates"""
    neighborhood: str = Field(description="The neighborhood in which the property is located.")
    price: NonNegativeInt = Field(description="The price of the property in USD.")
    bedrooms: NonNegativeInt = Field(description="The number of bedrooms in the property.")
    bathrooms: NonNegativeInt = Field(description="The number of bathrooms in the property.")
    house_size: NonNegativeInt = Field(description="The size of the property in sqft.")
    description: str = Field(description="A brief description of the property highlighting all its pros.")
    neighborhood_description: str = Field(description="A brief description of the neighborhood highlighting all its pros.")

In [47]:
class Listings(BaseModel):
    """The list containing real estates"""
    listings: List[RealEstate] = Field(description="A list containing Real Estates.")

In [48]:
# Initialize the parser
parser = PydanticOutputParser(pydantic_object=Listings)

In [49]:
# Create the prompt template
prompt_template = PromptTemplate(
    template="{instruction}\n{sample}\n{format_instructions}",
    input_variables=["instruction", "sample"],
    partial_variables={"format_instructions": parser.get_format_instructions}
)

print(prompt_template)

input_variables=['instruction', 'sample'] partial_variables={'format_instructions': <bound method PydanticOutputParser.get_format_instructions of PydanticOutputParser(pydantic_object=<class '__main__.Listings'>)>} template='{instruction}\n{sample}\n{format_instructions}'


In [50]:
# Create the query to generate the CSV table
query_to_generate_csv = prompt_template.format(
    instruction=INSTRUCTION,
    sample=SAMPLE_LISTING
)

print(query_to_generate_csv)

Generate a CSV file with at least 10 real estate listing.

Neighborhood: Green Oaks
Price: $800,000
Bedrooms: 3
Bathrooms: 2
House Size: 2,000 sqft

Description: Welcome to this eco-friendly oasis nestled in the heart of Green Oaks. This charming 3-bedroom, 2-bathroom home boasts energy-efficient features such as solar panels and a well-insulated structure. Natural light floods the living spaces, highlighting the beautiful hardwood floors and eco-conscious finishes. The open-concept kitchen and dining area lead to a spacious backyard with a vegetable garden, perfect for the eco-conscious family. Embrace sustainable living without compromising on style in this Green Oaks gem.

Neighborhood Description: Green Oaks is a close-knit, environmentally-conscious community with access to organic grocery stores, community gardens, and bike paths. Take a stroll through the nearby Green Oaks Park or grab a cup of coffee at the cozy Green Bean Cafe. With easy access to public transportation and bik

In [51]:
# LLM Response
llm_response = openai.ChatCompletion.create(
    model=MODEL,
    temperature=0.0,
    messages=[
        {
            "role": "user",
            "content": query_to_generate_csv
        }
    ]
)

In [52]:
resp_content = llm_response.choices[0].message.content
parsed_resp = parser.parse(resp_content)
print(parsed_resp)

listings=[RealEstate(neighborhood='Green Oaks', price=800000, bedrooms=3, bathrooms=2, house_size=2000, description='Welcome to this eco-friendly oasis nestled in the heart of Green Oaks. This charming 3-bedroom, 2-bathroom home boasts energy-efficient features such as solar panels and a well-insulated structure. Natural light floods the living spaces, highlighting the beautiful hardwood floors and eco-conscious finishes. The open-concept kitchen and dining area lead to a spacious backyard with a vegetable garden, perfect for the eco-conscious family. Embrace sustainable living without compromising on style in this Green Oaks gem.', neighborhood_description='Green Oaks is a close-knit, environmentally-conscious community with access to organic grocery stores, community gardens, and bike paths. Take a stroll through the nearby Green Oaks Park or grab a cup of coffee at the cozy Green Bean Cafe. With easy access to public transportation and bike lanes, commuting is a breeze.'), RealEstat

In [53]:
listings = parsed_resp.listings
df = pd.DataFrame([listing.dict() for listing in listings])
df.head()

,neighborhood,price,bedrooms,bathrooms,house_size,description,neighborhood_description
0,Green Oaks,800000,3,2,2000,Welcome to this eco-friendly oasis nestled in ...,"Green Oaks is a close-knit, environmentally-co..."
1,Sunnyvale,950000,4,3,2500,"Beautiful 4-bedroom, 3-bathroom home located i...","Sunnyvale offers top-rated schools, parks, and..."
2,Brooklyn Heights,1200000,5,4,3500,"Luxurious 5-bedroom, 4-bathroom home in the pr...",Brooklyn Heights is known for its historic bro...
3,Pacific Palisades,3500000,6,5,5000,"Magnificent 6-bedroom, 5-bathroom estate in th...",Pacific Palisades offers a relaxed coastal lif...
4,Upper West Side,1800000,4,3,2800,"Charming 4-bedroom, 3-bathroom brownstone in t...",The Upper West Side is known for its cultural ...


In [54]:
df.to_csv("Real_Estate.csv")

# TASK 2: CREATE THE VECTOR DATABASE USING TF-IDF

In [55]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

In [56]:
df = pd.read_csv("Real_Estate.csv")

In [57]:
def row_to_doc(row):
    return " ".join([
        f"""
        \nNeighborhood: {row["neighborhood"]},
        \nPrice: {row["price"]},
        \nBedrooms: {row["bedrooms"]},
        \nBathrooms: {row["bathrooms"]},
        \nHouse Size: {row["house_size"]}
        \nDescription: {row["description"]}
        \nNeighborhood Description: {row["neighborhood_description"]}
        """
    ])

In [58]:
docs_from_csv = []
for _, row in df.iterrows():
    docs_from_csv.append(row_to_doc(row))
    
print(docs_from_csv[0])


        
Neighborhood: Green Oaks,
        
Price: 800000,
        
Bedrooms: 3,
        
Bathrooms: 2,
        
House Size: 2000
        
Description: Welcome to this eco-friendly oasis nestled in the heart of Green Oaks. This charming 3-bedroom, 2-bathroom home boasts energy-efficient features such as solar panels and a well-insulated structure. Natural light floods the living spaces, highlighting the beautiful hardwood floors and eco-conscious finishes. The open-concept kitchen and dining area lead to a spacious backyard with a vegetable garden, perfect for the eco-conscious family. Embrace sustainable living without compromising on style in this Green Oaks gem.
        
Neighborhood Description: Green Oaks is a close-knit, environmentally-conscious community with access to organic grocery stores, community gardens, and bike paths. Take a stroll through the nearby Green Oaks Park or grab a cup of coffee at the cozy Green Bean Cafe. With easy access to public transportation and bike

In [59]:
vectorizer = TfidfVectorizer(stop_words="english")
tfidf = vectorizer.fit_transform(docs_from_csv)

# TASK 3: TAKE USER PREFERENCES

In [60]:
from typing import Optional

In [61]:
class Prefs:
    def __init__(self, budget, bedrooms, bathrooms, min_area, preferences):
        self.budget: Optional[int] = budget or None
        self.bedrooms: Optional[int] = bedrooms or None
        self.bathrooms: Optional[int] = bathrooms or None
        self.min_area: Optional[int] = min_area or None
        self.preferences: Optional[int] = preferences or ""

In [62]:
print("Let us know your preferences!")
budget = int(input("What's your budget? "))
bedrooms = int(input("How many bedrooms do you want in the property? "))
bathrooms = int(input("How many bathrooms do you want in the property? "))
min_area = int(input("How large of a property do you want? "))
preferences = input("Any preferences such as neighborhood? ")

Let us know your preferences!
What's your budget? 800000
How many bedrooms do you want in the property? 3
How many bathrooms do you want in the property? 2
How large of a property do you want? 2000
Any preferences such as neighborhood? no


In [63]:
prefs = Prefs(
    budget=budget,
    bedrooms=bedrooms,
    bathrooms=bathrooms,
    min_area=min_area,
    preferences=preferences
)

# TASK 4: SEMANTIC SEARCH

In [64]:
pref_query = " ".join([
    f"${prefs.budget}" if prefs.budget else "",
    f"{prefs.bedrooms} bedrooms" if prefs.bedrooms else "",
    f"{prefs.bathrooms} bathrooms" if prefs.bathrooms else "",
    f"{prefs.min_area} sqft" if prefs.min_area else "",
    prefs.preferences
])

print(pref_query)

$800000 3 bedrooms 2 bathrooms 2000 sqft no


In [66]:
query_vec = vectorizer.transform([pref_query])

In [67]:
scores = cosine_similarity(query_vec, tfidf)[0]

In [68]:
df["similarity_score"] = scores
ranked_df = df.sort_values(by="similarity_score", ascending=False)
ranked_df.head()

,Unnamed: 0,neighborhood,price,bedrooms,bathrooms,house_size,description,neighborhood_description,similarity_score
0,0,Green Oaks,800000,3,2,2000,Welcome to this eco-friendly oasis nestled in ...,"Green Oaks is a close-knit, environmentally-co...",0.123798
9,9,Malibu,6000000,5,4,4000,"Contemporary 5-bedroom, 4-bathroom beachfront ...","Malibu is known for its pristine beaches, cele...",0.031271
6,6,Georgetown,1600000,4,4,3000,"Elegant 4-bedroom, 4-bathroom townhouse in the...",Georgetown is known for its cobblestone street...,0.030866
8,8,Lakeview,900000,4,3,2600,"Spacious 4-bedroom, 3-bathroom home in the des...",Lakeview offers a vibrant community with trend...,0.030384
1,1,Sunnyvale,950000,4,3,2500,"Beautiful 4-bedroom, 3-bathroom home located i...","Sunnyvale offers top-rated schools, parks, and...",0.029679


# TASK 5: FILTERING

In [70]:
filtered = []

for _, row in ranked_df.iterrows():
    is_ok = True
    
    price, bedrooms, bathrooms, house_size = int(row["price"]), int(row["bedrooms"]), int(row["bathrooms"]), int(row["house_size"])
    
    if prefs.budget and prefs.budget < price:
        is_ok = False
    if prefs.bedrooms and prefs.bedrooms > bedrooms:
        is_ok = False
    if prefs.bathrooms and prefs.bathrooms > bathrooms:
        is_ok = False
    if prefs.min_area and prefs.min_area > house_size:
        is_ok = False
    
    if is_ok:
        filtered.append(row)
        
if not filtered:
    print("No Matches Found!")

# TASK 6: PERSONALIZED RECOMMENDATION USING LLM

### Not using System Prompt

In [71]:
final_prompt = ""

if filtered:
    best = filtered[0].drop("similarity_score").to_dict()
    
    final_prompt = f"""
    Here are the details of a property that is the closest to a buyer's preferences.
    
    {best}
    
    ---
    
    You are LLMRealtor, an experienced Real Estate Agent. 
    Your task is to convince a buyer that this property is the best for them.
    Use a warm and polite tone.
    Lay out in details all the positive features of this property.
    """
    
print(final_prompt)


    Here are the details of a property that is the closest to a buyer's preferences.
    
    {'Unnamed: 0': 0, 'neighborhood': 'Green Oaks', 'price': 800000, 'bedrooms': 3, 'bathrooms': 2, 'house_size': 2000, 'description': 'Welcome to this eco-friendly oasis nestled in the heart of Green Oaks. This charming 3-bedroom, 2-bathroom home boasts energy-efficient features such as solar panels and a well-insulated structure. Natural light floods the living spaces, highlighting the beautiful hardwood floors and eco-conscious finishes. The open-concept kitchen and dining area lead to a spacious backyard with a vegetable garden, perfect for the eco-conscious family. Embrace sustainable living without compromising on style in this Green Oaks gem.', 'neighborhood_description': 'Green Oaks is a close-knit, environmentally-conscious community with access to organic grocery stores, community gardens, and bike paths. Take a stroll through the nearby Green Oaks Park or grab a cup of coffee at the co

In [72]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    temperature=0.7,
    messages=[
        {
            "role": "user",
            "content": final_prompt
        }
    ]
)

In [73]:
print(response.choices[0].message.content)

Hello there!

I am thrilled to introduce you to this incredible property in Green Oaks that I truly believe is a perfect fit for you. Let me walk you through all the amazing features this home has to offer:

1. Eco-Friendly Oasis: This charming 3-bedroom, 2-bathroom home is designed with energy-efficiency in mind, featuring solar panels and a well-insulated structure. Embrace sustainable living without compromising on style.

2. Natural Light and Hardwood Floors: The living spaces are flooded with natural light, showcasing the beautiful hardwood floors and eco-conscious finishes that add a touch of elegance to the home.

3. Spacious Backyard with Vegetable Garden: The open-concept kitchen and dining area lead to a spacious backyard with a vegetable garden, perfect for those who appreciate growing their own produce and enjoying outdoor living.

4. Close-Knit Community: Green Oaks is a close-knit, environmentally-conscious community with access to organic grocery stores, community garden

### Using a System Prompt

In [74]:
system_prompt = """
You are LLMRealtor, a highly experienced Real Estate Agent who has sold over hundred real estates.
You are warm, polite and very convincing.
You will be given the best matching Real Estate according to a buyer's preference.
Your task will be to convince the buyer that this property is the best for them.
"""

In [75]:
final_prompt = ""

if filtered:
    best = filtered[0].drop("similarity_score").to_dict()
    
    final_prompt = f"""
    Here are the details of a property closest to a buyer's preferences:
    
    {best}
    
    ---
    
    Convince them to buy the property.
    """
    
print(final_prompt)


    Here are the details of a property closest to a buyer's preferences:
    
    {'Unnamed: 0': 0, 'neighborhood': 'Green Oaks', 'price': 800000, 'bedrooms': 3, 'bathrooms': 2, 'house_size': 2000, 'description': 'Welcome to this eco-friendly oasis nestled in the heart of Green Oaks. This charming 3-bedroom, 2-bathroom home boasts energy-efficient features such as solar panels and a well-insulated structure. Natural light floods the living spaces, highlighting the beautiful hardwood floors and eco-conscious finishes. The open-concept kitchen and dining area lead to a spacious backyard with a vegetable garden, perfect for the eco-conscious family. Embrace sustainable living without compromising on style in this Green Oaks gem.', 'neighborhood_description': 'Green Oaks is a close-knit, environmentally-conscious community with access to organic grocery stores, community gardens, and bike paths. Take a stroll through the nearby Green Oaks Park or grab a cup of coffee at the cozy Green Bea

In [76]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    temperature=0.7,
    messages=[
        {
            "role": "system",
            "content": system_prompt
        },
        {
            "role": "user",
            "content": final_prompt
        }
    ]
)

In [77]:
print(response.choices[0].message.content)

Hello and welcome! I am thrilled to present to you this incredible eco-friendly oasis in the heart of Green Oaks that perfectly aligns with your values and preferences. Let me paint a picture of why this property is truly a gem and why it is the best choice for you:

1. **Eco-Friendly Features:** This charming 3-bedroom, 2-bathroom home is equipped with energy-efficient features such as solar panels and a well-insulated structure. You can embrace sustainable living while enjoying lower energy costs and reducing your carbon footprint.

2. **Natural Light and Beautiful Finishes:** Natural light floods the living spaces, accentuating the gorgeous hardwood floors and eco-conscious finishes throughout the house. You will feel a sense of warmth and brightness as soon as you step inside.

3. **Spacious Backyard with Vegetable Garden:** The open-concept kitchen and dining area lead to a spacious backyard with a vegetable garden. Imagine harvesting your own fresh produce right at your doorstep 